<a href="https://colab.research.google.com/github/jasdeepbajaj/Image_Captioning/blob/main/Image_Captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("hello world")
print("hello world")

hello world
hello world
